So in this notebook i want to try and act as a like searchbar in a shop where you would get suggestions without having to type in the full word.

I proberbly have to try and use different tokenizers at indexing and search time

I proberbly have to include fuzzing for spelling mistakes

and maybe more

In [ ]:
from helpers.es.es_setup_methods import *
from helpers.es.es_helper_methods import *

document_amount = 2_000_000

standard_index = "standard_index"

edge_ngram_index = "edge_ngram_index"

So first, Lets try with standard everything and see what it can do by default

In [ ]:
mapping = {
    "mappings": {
        "properties": {
            "index": {
                "type": "integer",
                "index": True
            },
            "name": {
                "type": "text",
            },
            "description": {
                "type": "text"
            },
            "brand": {
                "type": "text"
            },
            "category": {
                "type": "keyword"
            },
            "price": {
                "type": "integer"
            },
            "stock": {
                "type": "integer"
            },
            "color": {
                "type": "keyword"
            },
            "size": {
                "type": "keyword"
            },
            "availability": {
                "type": "keyword"
            }
        }
    }
}

settings = {
    "settings": {
        "analysis": {
            "analyzer": {
                "standard": {
                    "type": "standard"
                },
            },
            "tokenizer": {
                "standard": {
                    "type": "standard",
                }
            }
        }
    }
}


init_index(index_name=standard_index,mapping=mapping, settings=settings, documents_records=document_amount)


print("sleeping to just make sure the index is ready")

time.sleep(2)

If we try to search for all wireless chargers just to use this as a benchmark performance. 
We could have made a search what would be just "name": "wireless charger" but this would return all hits where there is EITHER wireless or charger in the name. So instead i use the query and and-operator just to specify that the names wireless and charger should be in the name (but this dont specify that they have to be next to each other)

In [ ]:
user_input = "wireless charger"

query = {
    "query": {
        "match": {
            "name": {
                "query": user_input,
                "operator": "and"
            }
        }
    }
}

search_text(query, standard_index)

Just for another sort of benchmark, the standard analyzer wont allow for auto completion

In [ ]:
user_input = "charg"

query = {
    "query": {
        "match": {
            "name": user_input
        }
    }
}

search_text(query, standard_index)

In [ ]:
mapping = {
    "mappings": {
        "properties": {
            "index": {
                "type": "integer",
                "index": True
            },
            "name": {
                "type": "text",
                "analyzer": "product_search_analyzer",
                "search_analyzer": "standard"
            },
            "description": {
                "type": "text"
            },
            "brand": {
                "type": "text"
            },
            "category": {
                "type": "keyword"
            },
            "price": {
                "type": "integer"
            },
            "stock": {
                "type": "integer"
            },
            "color": {
                "type": "keyword"
            },
            "size": {
                "type": "keyword"
            },
            "availability": {
                "type": "keyword"
            }
        }
    }
}

settings = {
    "settings": {
        "analysis": {
            "analyzer": {
                "product_search_analyzer": {
                    "tokenizer": "my_edge_ngram",
                    "filter": [
                        "lowercase"
                    ]
                }
            },
            "tokenizer": {
                "my_edge_ngram": {
                    "type": "edge_ngram",
                    "min_gram": 1,
                    "max_gram": 20,
                    "token_chars": [
                        "letter",
                        "digit"
                    ]
                }
            }
        }
    }
}


init_index(index_name=edge_ngram_index, mapping=mapping, settings=settings, documents_records=document_amount)

print("sleeping to just make sure the index is ready")

time.sleep(2)

Here we have made a edge_gram index and this should allow us to now partly spell words. But lets first get the benchmark of just wireless charger

In [ ]:
user_input = "wireless charger"

query = {
    "query": {
        "match": {
            "name": {
                "query": user_input,
                "operator": "and"
            }
        }
    }
}

search_text(query, edge_ngram_index)


The benchmark got around the same times. So the egde ngram doesn't bring speed to the table. And i think it would somewhat do the oppisite since it would make the tokenization part take a bit longer i would assume. And it would also possbile add a bit of extra storage to store all of these extra tokens. 
I dont quite know like the price of the cost if that makes sense

In [ ]:
user_input = "wir charg"

query = {
    "query": {
        "match": {
            "name": {
                "query": user_input,
                "operator": "and"
            }
        }
    }
}

search_text(query, edge_ngram_index)


This just showed us that now we can search of part of the words instead of having to type the full words. And it is the same performance as the full word search